**Inference к модели, захосченной через ngrok & LM Studio**

In [36]:
import requests
import json

def message_llm(system_prompt, user_input, temperature=0.7):
    '''Function to message (test) our LLM'''

    try:
        url = "https://live-relaxed-oryx.ngrok-free.app/v1/chat/completions"

        data = {
          "messages": [
            { "role": "system", "content": system_prompt},
            { "role": "user", "content": user_input}
          ],
          "temperature": 0.7,
          "max_tokens": -1,
          "stream": False
        }

        headers = {
            "Content-Type": "application/json"
        }

        response = requests.post(url, headers=headers, data=json.dumps(data))

        if response.status_code == 200:
            return eval(response.text.replace("\n  ", ""))['choices'][0]['message']['content']
        else:
            print("Ошибка запроса:\n", response.status_code, response.text)
            return ""
            
    except Exception as e:
        
        print(f"Error {e}!")
        
batch = "Банк России поддерживает инфляцию вблизи целевого уровня — 4%. \
Такая политика называется таргетированием инфляции (от англ. target — «цель»). \
Основной инструмент денежно-кредитной политики — это ключевая ставка. \
Ключевая ставка — это процент, под который центральный банк выдает кредиты коммерческим банкам \
и принимает от них деньги на депозиты. Изменение ключевой ставки влияет на спрос через ставки \
в экономике и в конечном итоге — на инфляцию. При этом центральный банк опирается на макроэкономический \
прогноз и учитывает ситуацию в экономике и на финансовых рынках."

rag_prompt = ""

system_prompt = f"На основании приведенного контекста дай исчерпывающий ответ на следующий вопрос \
(если ответа на вопрос нет, направь пользователя на главную страницу Центробанка РФ: https://cbr.ru/): \
КОНТЕКСТ: {batch}"

user_prompt = "На каком уровне ЦБ РФ поддерживает ставку инфляции? На что опирается ЦБ?"

answer = message_llm(system_prompt, user_prompt)

In [37]:
answer

'ЦБ РФ поддерживает ставку инфляции на уровне 4%. Центральный банк опирается на макроэкономический прогноз и учитывает ситуацию в экономике и на финансовых рынках.'